# <div align="center" style="color: #ff5733;">TSA Closure Third Notification</div>

In [1]:
# Import Libraries   

import pandas as pd  
import numpy as np  

from google.cloud import bigquery
import os

path = r"C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json"

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = path
client = bigquery.Client(project="prj-prod-dataplatform")

# Ask Balasubramani to refresh risk_credit_mis.tsa_account_closure table.
**Check if that is refreshed when Balasubramani confirms it**

In [2]:
sq = """select Run_Date, count(1)cnt from risk_credit_mis.tsa_account_closure group by 1 order by 1 desc"""
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')
df.head(1)

Job ID 7f6567f8-d9a1-4a87-a4e6-2cd73e9090a1 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,Run_Date,cnt
0,2024-12-25,747381


The date should show yesterday's date

# Read the data from the first sheet for Type1 and Type2 data provided by Marvin

In [3]:
d1 = pd.read_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Marvin\TSA Account Closure\November_December2024\ThirdNotification_files\TYPE 1 & 2.xlsx")
d1.shape

(59628, 2)

In [4]:
print(f"The unique count of customer_id is:\t{len(d1['Customer_Id'].unique())}")
d1.head()


The unique count of customer_id is:	59628


,Customer_Id,ACTION_TYPE
0,2853226,TYPE 1
1,2857135,TYPE 1
2,2801829,TYPE 1
3,2691958,TYPE 1
4,2719103,TYPE 1


In [5]:
d1.groupby('ACTION_TYPE')['Customer_Id'].nunique()

ACTION_TYPE
TYPE 1    59574
TYPE 2       54
Name: Customer_Id, dtype: int64

# Read the data from the second sheet for Type3 data provided by Marvin

In [6]:
d2 = pd.read_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Marvin\TSA Account Closure\November_December2024\ThirdNotification_files\tYPE 3.xlsx")
d2.shape

(1468, 2)

In [7]:
print(f"The unique count of customer_id is:\t{len(d2['Customer_Id'].unique())}")
d2.head()

The unique count of customer_id is:	1468


,Customer_Id,ACTION_TYPE
0,1048271,TYPE 3
1,2493127,TYPE 3
2,1014850,TYPE 3
3,1087456,TYPE 3
4,1083549,TYPE 3


In [8]:
d2.groupby('ACTION_TYPE')['Customer_Id'].nunique()

ACTION_TYPE
TYPE 3    1468
Name: Customer_Id, dtype: int64

In [9]:
result = pd.concat([d1, d2], axis=0, ignore_index=True)
result.shape

(61096, 2)

In [10]:
result.ACTION_TYPE.value_counts()

ACTION_TYPE
TYPE 1    59574
TYPE 3     1468
TYPE 2       54
Name: count, dtype: int64

In [14]:
result[result['Customer_Id'] == '2493127']

,Customer_Id,ACTION_TYPE
59629,2493127,TYPE 3


In [12]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61096 entries, 0 to 61095
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Customer_Id  61096 non-null  int64 
 1   ACTION_TYPE  61096 non-null  object
dtypes: int64(1), object(1)
memory usage: 954.8+ KB


In [13]:
result['Customer_Id'] = result['Customer_Id'].astype(str)
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61096 entries, 0 to 61095
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Customer_Id  61096 non-null  object
 1   ACTION_TYPE  61096 non-null  object
dtypes: object(2)
memory usage: 954.8+ KB


In [15]:
result.to_csv("temp.csv", index = False)

In [16]:
a = tuple(result['Customer_Id'])
a[:5]

('2853226', '2857135', '2801829', '2691958', '2719103')

In [17]:
d = '2024-12-25'   ### Need to change this to D-1 everytime we run this code ###
# Get the current date
current_date = pd.Timestamp.now().date()
d_minus_1_days = current_date - pd.DateOffset(days=1)
# Calculate D-60 days
d_minus_60_days = '2024-09-26'
# Calculate D-180 days
d_minus_180_days = '2024-05-29'

print("D-1 day from current date", d_minus_1_days )
print("D-60 day from the current date - 1:", d_minus_60_days)
print("D-180 day from the current date -1:", d_minus_180_days)

d_minus_1_days_string = d_minus_1_days.strftime('%Y-%m-%d')
print("D-1 day from the current date:", d_minus_1_days_string)

d_minus_60_days_string = d_minus_60_days
print("D-60 day from the current date:", d_minus_60_days_string)

d_minus_180_days_string = d_minus_180_days
print("D-180 day from the current date:", d_minus_180_days_string)

D-1 day from current date 2024-12-25 00:00:00
D-60 day from the current date - 1: 2024-09-26
D-180 day from the current date -1: 2024-05-29
D-1 day from the current date: 2024-12-25
D-60 day from the current date: 2024-09-26
D-180 day from the current date: 2024-05-29


Step1: At line 11 change the date to D-1<br><br>
Step2: At line around 36 (and (TSA_Opening_date < date '2023-10-18')   ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) Change the date D-60 Days <br><br>
Step3: At line around 45 (WHEN tsa_status = 'Active' and Last_TSA_EOD_Balance_Date is null and (TSA_Opening_date < date '2023-10-18')  ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) Change the date D-60 Days <br><br>
Step4: At line 58 (and TSA_Opening_date < date '2023-06-18'  and (Last_TSA_Credit_Date < date '2023-06-18' and Last_TSA_Debit_Date < date '2023-06-18' and coalesce(Last_Group_Stash_Credit_Date,date '1970-01-01') < date '2023-06-18')  ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) <br><br>
Step5: At line 72 (Run_Date = date '2024-01-09'###DATE change here to D-1) Change the date to D-1<br><br>

First Notification we change all the steps <br><br>
Second and Third Notification we only change Step1 and Step5
For Second Notificaiton and Third Notificaiton the customre id should be from the file sent by Marvin. <br><br>

In [18]:
sq = f"""
with
w_kyc
as
(
  select distinct PARTYID, SEGCATEGORYDESC, row_number() over(partition by partyid order by SEGEFFECTIVEFROM desc) as rnk from `core_raw.party_details`
),
w_tsa
as
(
  select * from risk_credit_mis.tsa_account_closure where Run_Date = date '{d_minus_1_days_string}' ###DATE change here to D-1
  and
  Customer_Id not in (select cast(PARTYID as string) from w_kyc where rnk = 1 and SEGCATEGORYDESC = 'BKYC')
  and
  --customer_id in (select cast(customer_id as string) from prj-prod-dataplatform.dap_ds_poweruser_playground.tsa_2nd_notification_base_10jan2024)
  customer_id in {a}
)
select * from w_tsa;
"""

dfd = client.query(sq).to_dataframe(progress_bar_type =  'tqdm')
dfd.shape

Job ID 7dd3098d-a3f1-4421-9e48-9fcea7577ade successfully executed: 100%|██████████|
Downloading: 100%|██████████|


(61095, 29)

In [19]:
sq = f"""
with
w_kyc
as
(
  select distinct PARTYID, SEGCATEGORYDESC, row_number() over(partition by partyid order by SEGEFFECTIVEFROM desc) as rnk from `core_raw.party_details`
),
w_tsa
as
(
  select * from risk_credit_mis.tsa_account_closure where Run_Date = date '{d_minus_1_days_string}'  ###DATE change here to D-1
  and
  Customer_Id not in (select cast(PARTYID as string) from w_kyc where rnk = 1 and SEGCATEGORYDESC = 'BKYC')
  and
  --customer_id in (select cast(customer_id as string) from prj-prod-dataplatform.dap_ds_poweruser_playground.tsa_2nd_notification_base_10jan2024)
  customer_id in {a}
),
w_approved
as
(
  select DISTINCT customerId from risk_credit_mis.loan_master_table where loantype = 'BIGLOAN' OR FLAGAPPROVAL = 1 OR FLAGDISBURSEMENT = 1  
  or (stage1Status = 'EXPIRED' AND cddInitiatedDateAndTime IS NULL)
)
select *
from
(
  select distinct
  a.Customer_Id,a.TSA_Opening_date,
  c.OFSTANDARDACCOUNTID as TSA_Account_Number, c.OfblockAmount, 
  a.TSA_Status,a.TSA_Balance,a.Solo_Stash_Balance,a.Group_Stash_Balance,a.Term_Deposit_Balance,a.Active_Term_Deposit_flag,a.Last_TSA_Credit_Date,a.Last_TSA_Debit_Date,
  a.ARI_Flag,a.Active_Loan,a.Total_Loan_Outstanding,  case when b.customerId is not null then 1 else 0 end as Approved_Loans,
  a.Pending_Loan_Applications,a.Hard_Reject_Loans_App,a.Soft_Reject_Loans_App,a.Cancelled_Loans_App, extract(date from a.Last_Login_Date) as Last_Login_Date,
  a.Last_TSA_EOD_Balance_Date,a.Last_Group_Stash_Credit_Date,a.Group_Stash_Balance_Owner,a.Run_Date,a.Batch,a.Processing_Month,a.pending_loans_not_expired_not_exempt,a.expired_loans,
  a.pending_loans_lastDt,a.expired_loans_lastDt,
  CASE 
    WHEN tsa_status = 'Active' and (Last_TSA_Credit_Date = date '1970-01-01' and Last_Group_Stash_Credit_Date is null)
    and (TSA_Opening_date < date '{d_minus_60_days_string}')   ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
    and ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) = 0 )
    and (coalesce(Active_Loan,0) = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
    and (Active_Term_Deposit_flag = 0)
    and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)  THEN 'TYPE 1'
    and b.customerId is null  THEN 'TYPE 1'


    WHEN tsa_status = 'Active' and Last_TSA_EOD_Balance_Date is null and (TSA_Opening_date < date '{d_minus_60_days_string}')  ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
    and ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) = 0 )
    and (coalesce(Active_Loan, 0) = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
    and (Active_Term_Deposit_flag = 0)
    and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)
    and b.customerId is null  
    and (date_trunc(TSA_Opening_date,day) = date_trunc(Last_Login_Date,day)) THEN 'TYPE 2'

    
    WHEN ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) < 1 )
            and (coalesce(Active_Loan, 0) = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
            and (Active_Term_Deposit_flag = 0)
            and TSA_Opening_date < date '{d_minus_180_days_string}'                                     ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
            and (Last_TSA_Credit_Date < date '{d_minus_180_days_string}'                                 ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
            and Last_TSA_Debit_Date < date '{d_minus_180_days_string}'                                         ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
            and coalesce(Last_Group_Stash_Credit_Date,date '1970-01-01') < date '{d_minus_180_days_string}' )  ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
            and tsa_status = 'Active'
            and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)
            and b.customerId is null  
    THEN 'TYPE 3'
    ELSE 'NO ACTION'
    END AS ACTION_TYPE
  from w_tsa A
  left join w_approved b
  on cast(a.Customer_Id as int64) = b.customerId
  left join core_raw.customer_accounts c
  on c.OFCUSTOMERID = a.Customer_Id and c.CRINTERDESC = 'Transactional Savings Account Inv_R' and c.OFISCLOSED = 'N'
  where
  Run_Date = date '{d_minus_1_days_string}'  ###DATE change here to D-1
)
"""


In [20]:
print(sq)


with
w_kyc
as
(
  select distinct PARTYID, SEGCATEGORYDESC, row_number() over(partition by partyid order by SEGEFFECTIVEFROM desc) as rnk from `core_raw.party_details`
),
w_tsa
as
(
  select * from risk_credit_mis.tsa_account_closure where Run_Date = date '2024-12-25'  ###DATE change here to D-1
  and
  Customer_Id not in (select cast(PARTYID as string) from w_kyc where rnk = 1 and SEGCATEGORYDESC = 'BKYC')
  and
  --customer_id in (select cast(customer_id as string) from prj-prod-dataplatform.dap_ds_poweruser_playground.tsa_2nd_notification_base_10jan2024)
  customer_id in ('2853226', '2857135', '2801829', '2691958', '2719103', '2797412', '2792698', '2881198', '2687131', '2732083', '2744958', '2768730', '2881202', '2700263', '2746290', '2827032', '2690291', '2697709', '2700407', '2721064', '2739843', '2662578', '2721444', '2719134', '2845286', '2851383', '2558805', '2761858', '2781185', '2663021', '2835122', '2734999', '2701956', '2768567', '2812486', '2685608', '2769629', '2806818',

In [21]:
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID 997617e3-0947-44a1-b2a7-34a719bbb89d successfully executed: |          |
Downloading: 100%|██████████|


In [22]:
pd.set_option('display.max_columns', None)
df.sample(10)

,Customer_Id,TSA_Opening_date,TSA_Account_Number,OfblockAmount,TSA_Status,TSA_Balance,Solo_Stash_Balance,Group_Stash_Balance,Term_Deposit_Balance,Active_Term_Deposit_flag,Last_TSA_Credit_Date,Last_TSA_Debit_Date,ARI_Flag,Active_Loan,Total_Loan_Outstanding,Approved_Loans,Pending_Loan_Applications,Hard_Reject_Loans_App,Soft_Reject_Loans_App,Cancelled_Loans_App,Last_Login_Date,Last_TSA_EOD_Balance_Date,Last_Group_Stash_Credit_Date,Group_Stash_Balance_Owner,Run_Date,Batch,Processing_Month,pending_loans_not_expired_not_exempt,expired_loans,pending_loans_lastDt,expired_loans_lastDt,ACTION_TYPE
53814,2684492,2024-08-25,60826844920002,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-08-25,NaT,NaT,NaN,2024-12-25,Batch_1,December,0,1,NaT,2024-10-22,TYPE 1
34021,2687518,2024-08-16,60826875180007,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,1,0,2024-12-09,NaT,NaT,NaN,2024-12-25,Batch_1,December,0,1,NaT,2024-10-23,TYPE 1
21795,2824643,2024-09-05,60828246430001,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-09-05,NaT,NaT,NaN,2024-12-25,Batch_1,December,0,0,NaT,NaT,TYPE 1
14709,2737537,2024-08-09,60827375370001,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-09-10,NaT,NaT,NaN,2024-12-25,Batch_1,December,0,1,NaT,2024-11-08,TYPE 1
23592,2714222,2024-08-02,60827142220003,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-08-02,NaT,NaT,NaN,2024-12-25,Batch_1,December,0,1,NaT,2024-11-01,TYPE 1
855,2809662,2024-09-17,60828096620004,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-10-28,NaT,NaT,NaN,2024-12-25,Batch_1,December,0,1,NaT,2024-11-30,TYPE 1
22426,2806211,2024-08-30,60828062110009,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-08-30,NaT,NaT,NaN,2024-12-25,Batch_1,December,0,1,NaT,2024-11-29,TYPE 1
2791,2760761,2024-08-16,60827607610004,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-08-17,NaT,NaT,NaN,2024-12-25,Batch_1,December,0,0,NaT,NaT,TYPE 1
19933,2854022,2024-09-17,60828540220003,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-09-17,NaT,NaT,NaN,2024-12-25,Batch_1,December,0,1,NaT,2024-12-17,TYPE 1
165,2688029,2024-07-25,60826880290009,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,1,0,2024-12-18,NaT,NaT,NaN,2024-12-25,Batch_1,December,0,1,NaT,2024-10-25,TYPE 1


In [23]:
df.shape

(61095, 32)

In [24]:
df['ACTION_TYPE'].value_counts()

ACTION_TYPE
TYPE 1       59460
TYPE 3        1462
NO ACTION      119
TYPE 2          54
Name: count, dtype: int64

In [25]:
df.to_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Marvin\TSA Account Closure\November_December2024\ThirdNotification_files\20241224_TSA_Account_Closure_3rdtNotification.xlsx", sheet_name='TSA_Closure_3rdNotification', index = False)

In [29]:
df = pd.read_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Marvin\TSA Account Closure\November_December2024\ThirdNotification_files\20241224_TSA_Account_Closure_3rdtNotification.xlsx", sheet_name='TSA_Closure_3rdNotification')
df.shape

(61095, 32)

In [30]:
df.columns

Index(['Customer_Id', 'TSA_Opening_date', 'TSA_Account_Number',
       'OfblockAmount', 'TSA_Status', 'TSA_Balance', 'Solo_Stash_Balance',
       'Group_Stash_Balance', 'Term_Deposit_Balance',
       'Active_Term_Deposit_flag', 'Last_TSA_Credit_Date',
       'Last_TSA_Debit_Date', 'ARI_Flag', 'Active_Loan',
       'Total_Loan_Outstanding', 'Approved_Loans', 'Pending_Loan_Applications',
       'Hard_Reject_Loans_App', 'Soft_Reject_Loans_App', 'Cancelled_Loans_App',
       'Last_Login_Date', 'Last_TSA_EOD_Balance_Date',
       'Last_Group_Stash_Credit_Date', 'Group_Stash_Balance_Owner', 'Run_Date',
       'Batch', 'Processing_Month', 'pending_loans_not_expired_not_exempt',
       'expired_loans', 'pending_loans_lastDt', 'expired_loans_lastDt',
       'ACTION_TYPE'],
      dtype='object')

In [31]:
df[df['ACTION_TYPE']!='NO ACTION'].groupby('ACTION_TYPE')['Customer_Id'].nunique()

ACTION_TYPE
TYPE 1    59460
TYPE 2       54
TYPE 3     1462
Name: Customer_Id, dtype: int64

In [32]:
59460+54+1462

60976

In [33]:
df.groupby('ACTION_TYPE')['Customer_Id'].count()

ACTION_TYPE
NO ACTION      119
TYPE 1       59460
TYPE 2          54
TYPE 3        1462
Name: Customer_Id, dtype: int64

In [34]:
119+59460+54+1462

61095

In [35]:
len(df['Customer_Id'].unique())

61095